## 0. Import Libraries

In [2]:
# ====================================================================
# STEP 0: Import Libraries
# ====================================================================
# 과제에 사용할 library들을 import

# 시스템 및 입출력 관련
import os  # 디렉토리, 파일 경로 조작 등
from PIL import Image  # 이미지 열기 및 처리 (Pillow)
from tqdm import tqdm  # 반복문의 진행 상태 시각화
from pathlib import Path  # payhon path


# 시각화 도구
import matplotlib.pyplot as plt  # 기본 시각화 라이브러리
import seaborn as sns  # 고급 시각화 (히트맵, 스타일 등)

# 이미지 처리
import cv2  # OpenCV - 고급 이미지/비디오 처리

# 수치 연산
import numpy as np  # 배열, 벡터 계산 등

# PyTorch 기본 구성
import torch  # 텐서, 연산 등
import torch.nn as nn  # 모델 정의 (layer, loss 등)
import torch.nn.functional as F
import torch.optim as optim  # Optimizer (SGD, Adam 등)

# PyTorch 데이터 처리
from torch.utils.data import Dataset, DataLoader  # 커스텀 데이터셋, 배치 로딩

# PyTorch 이미지 전처리
import torchvision
from torchvision import transforms  # 기본 이미지 transform
from torchvision import datasets  # torchvision 내장 데이터셋
import torchvision.models as models

from torchvision.transforms import v2  # torchvision v2 transforms (최신 API)

# 싸이킷런 평가 지표
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# 싸이킷런 데이터 나누기
from sklearn.model_selection import train_test_split

# 통계 tool
import pandas as pd

# 실험 추적 및 하이퍼파라미터 관리
import wandb  # Weights & Biases - 실험 로깅, 시각화, 하이퍼파라미터 튜닝

# Garbage Collector 모듈
import gc

# Data Augmentation 패키지: Albumentations
import albumentations as A

## 1. Set configuration

In [3]:
# ====================================================================
# STEP 1: Configuration 설정
# ====================================================================
# 하이퍼파라미터 및 경로 등 실험에 필요한 설정들을 모아둠
# 실험 추적 및 재현성을 위해 모든 값은 여기에서 수정하고자 함

# 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 주요 하이퍼파라미터
LEARNING_RATE = 1e-4  # 학습률 (optimizer용)
BATCH_SIZE = 16  # 배치 크기
NUM_EPOCHS = 100  # 학습 epoch 수
SEED = 42  # 재현성을 위한 random seed

# 데이터 경로 설정
# DATA_ROOT = path
# train_dir = os.path.join(DATA_ROOT, "train")
# val_dir = os.path.join(DATA_ROOT, "val")
# test_dir = os.path.join(DATA_ROOT, "test")

# 모델 설정
MODEL_NAME = "PROJECT1_baseline"  # 또는 "EfficientNet", 등등
USE_PRETRAINED = True  # torchvision 모델 사용 여부

# 학습 고도화 설정 (Optional)
USE_SCHEDULER = True  # Learning rate scheduler 사용 여부
EARLY_STOPPING = True  # Early stopping 적용 여부
AUGMENTATION = True  # 데이터 증강 사용 여부

# 실험 로깅용 설정
USE_WANDB = True
WANDB_PROJECT = "cats-and-dogs-breeds-classification-oxford-dataset"
RUN_NAME = f"{MODEL_NAME}_bs{BATCH_SIZE}_lr{LEARNING_RATE}"

Using device: cuda


## 2. Data pre-processing

In [4]:
# ====================================================================\n",
# STEP 2: Data Pre-processing\n",
# ====================================================================\n",

# 데이터셋 경로 (수정 필요)
DATA_ROOT = "../data/raw/ai03-level1-project/"  # 예시 경로
train_image_dir = os.path.join(DATA_ROOT, "train_images")
train_ann_path = os.path.join(DATA_ROOT, "train_annotations")
test_image_dir = os.path.join(DATA_ROOT, "test_images")

In [5]:
train_image_dir

'../data/raw/ai03-level1-project/train_images'

In [ ]:
import json

# 사용자 정의 데이터셋 클래스
class PillDataset(Dataset):
    def __init__(self, image_dir, annotation_path, transforms=None):
        self.image_dir = Path(image_dir)
        self.transforms = transforms
        
        # 어노테이션 파일 로드
        with open(annotation_path) as f:
            self.annotations = json.load(f)
            
        self.image_info = self.annotations['images']
        self.annotation_info = self.annotations['annotations']
        self.category_info = self.annotations['categories']
        
        # 이미지 ID를 키로 어노테이션을 빠르게 찾기 위한 딕셔너리
        self.image_id_to_anns = {}
        for ann in self.annotation_info:
            img_id = ann['image_id']
            if img_id not in self.image_id_to_anns:
                self.image_id_to_anns[img_id] = []
            self.image_id_to_anns[img_id].append(ann)

    def __len__(self):
        return len(self.image_info)

    def __getitem__(self, idx):
        # 이미지 정보 가져오기
        img_info = self.image_info[idx]
        image_path = self.image_dir / img_info['file_name']
        
        # 이미지 로드 (Albumentations는 BGR 순서를 사용)
        image = cv2.imread(str(image_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 해당 이미지의 모든 어노테이션 가져오기
        image_id = img_info['id']
        anns = self.image_id_to_anns.get(image_id, [])
        
        boxes = []
        labels = []
        
        for ann in anns:
            # bbox: [x, y, w, h] -> [xmin, ymin, xmax, ymax]
            x, y, w, h = ann['bbox']
            boxes.append([x, y, x + w, y + h])
            labels.append(ann['category_id'])

        # torchvision 모델이 요구하는 target 형태
        target = {
            'boxes': torch.as_tensor(boxes, dtype=torch.float32),
            'labels': torch.as_tensor(labels, dtype=torch.int64)
        }

        # Albumentations 변환 적용
        if self.transforms:
            transformed = self.transforms(image=image, bboxes=target['boxes'], labels=target['labels'])
            image = transformed['image']
            # Albumentations는 bbox 형식을 유지하므로 변환 필요 없음
            target['boxes'] = torch.as_tensor(transformed['bboxes'], dtype=torch.float32)
            # 만약 bbox가 augmentation 후 사라지면 에러가 날 수 있으므로 처리
            if len(target['boxes']) == 0:
                # 임의의 작은 박스를 넣어주거나, 이 이미지를 스킵하는 로직이 필요
                # 여기서는 간단하게 크기가 0인 텐서를 만듦
                target['boxes'] = torch.zeros((0, 4), dtype=torch.float32)


        return image, target


In [ ]:

# 데이터 증강 (Augmentation) 정의
# Albumentations를 사용하는 것이 바운딩 박스 변환에 더 유리해
train_transforms = A.Compose([
    A.Resize(512, 512),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    # PyTorch 텐서로 변환
    A.pytorch.ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels'])) # bbox 형식은 pascal_voc: [xmin, ymin, xmax, ymax]

test_transforms = A.Compose([
    A.Resize(512, 512),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    A.pytorch.ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))


# 데이터셋 인스턴스 생성
# train_dataset = PillDataset(train_image_dir, train_ann_path, transforms=train_transforms)
# val_dataset = PillDataset(val_image_dir, val_ann_path, transforms=val_transforms)

# DataLoader를 위한 collate_fn. 이미지와 타겟을 리스트로 묶어줌
def collate_fn(batch):
    return tuple(zip(*batch))

# 데이터 로더 생성 (실제 데이터셋 경로 설정 후 주석 해제)
# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
# val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# print("DataLoaders are ready.")
# print(f"Number of training samples: {len(train_dataset)}")
# print(f"Number of validation samples: {len(val_dataset)}")


## 3. Model implementation

## 4. Train and Evaluate models

## 5. Train and Evaluate a model

## 6. Results & Disscussion

## 7. Conclusion